# **Project Summary: Building a Shakespeare Search Engine**

In this project, we set out to create a search engine for exploring the works of William Shakespeare. The dataset consisted of 44 Shakespearean books, and our goal was to construct a tool that allows users to search for specific words or combinations of words within these literary works.

**Key Steps:**

**Indexing the Books:**

We began by creating a binary search tree, a data structure that helps organize and retrieve words from the books efficiently. Each node in the tree represented a unique word, and the associated documents were stored in a list.

**Text Normalization:**

To enhance search accuracy, we applied normalization techniques, including case folding (making all text lowercase), removing stop words (common words like "the," "and," etc.), lemmatization (reducing words to their base form), and stemming (reducing words to their root).

**Query Parsing and Execution:**

Users can input queries, combining words with operators like OR and AND. We implemented a query parser to interpret these queries, considering parentheses and applying the specified operators. The search engine then retrieved documents matching the query criteria.

**Handling Missing Words:**

In cases where a word wasn't found in the tree, we implemented a mechanism to suggest the closest matching word using the Levenshtein algorithm (edit distance).

**Query Rules and Result Display:**

The search engine supports both AND and OR operations in queries, allowing users to refine their searches. The results, along with the elapsed time for each query, are presented to the user.

In [1]:
import os
import zipfile

# Define the path to the zip file and the extraction directory
zip_file_path = "/content/BOOKS.zip"  # Update with the correct path
extraction_dir = "/content/extracted_books"

# Unzip the books
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# List all text files in the extraction directory (inside the 'BOOKS' folder)
books_folder_path = os.path.join(extraction_dir, 'BOOKS')
shakespeare_books = [os.path.join(books_folder_path, file) for file in os.listdir(books_folder_path) if file.endswith(".txt")]

# Confirm the list of books
print("List of Shakespeare books:")
print(shakespeare_books)

List of Shakespeare books:
['/content/extracted_books/BOOKS/32. THE TEMPEST.txt', '/content/extracted_books/BOOKS/9. THE FIRST PART OF KING HENRY THE FOURTH.txt', '/content/extracted_books/BOOKS/16. THE LIFE AND DEATH OF KING JOHN.txt', '/content/extracted_books/BOOKS/44. VENUS AND ADONIS.txt', '/content/extracted_books/BOOKS/41. THE PASSIONATE PILGRIM.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/39. THE WINTERS TALE.txt', '/content/extracted_books/BOOKS/33. THE LIFE OF TIMON OF ATHENS.txt', '/content/extracted_books/BOOKS/26. THE TRAGEDY OF OTHELLO THE MOOR OF VENICE.txt', '/content/extracted_books/BOOKS/28. KING RICHARD THE SECOND.txt', '/content/extracted_books/BOOKS/17. THE TRAGEDY OF JULIUS CAESAR.txt', '/content/extracted_books/BOOKS/24. A MIDSUMMER NIGHTS DREAM.txt', '/content/extracted_books/BOOKS/5. THE COMEDY OF ERRORS.txt', '/content/extracted_books/BOOKS/30. THE TRAGEDY OF ROMEO AND JULIET.txt', '/content/extracted_b

In [2]:
!pip install nltk

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('wordnet')

def normalize_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Case folding: Convert to lowercase
    words = [word.lower() for word in words]

    # Remove punctuation
    words = [word.strip(string.punctuation) for word in words if word.isalnum()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    return words


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Function to construct a binary search tree for indexing
from nltk.tokenize import word_tokenize

class TreeNode:
    def __init__(self, term, books):
        self.term = term
        self.books = books
        self.left = None
        self.right = None

def construct_bst(books):
    # Initialize the root of the BST
    root = None

    for book_path in books:
        with open(book_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Tokenize and normalize the words
        words = normalize_text(content)

        # Update the BST for each word in the book
        for word in words:
            root = insert_word(root, word, book_path)

    return root

def insert_word(root, word, book_path):
    if root is None:
        return TreeNode(word, [book_path])

    # Compare the word with the current node's term
    if word < root.term:
        root.left = insert_word(root.left, word, book_path)
    elif word > root.term:
        root.right = insert_word(root.right, word, book_path)
    else:
        # If the word already exists, update the list of books
        root.books.append(book_path)

    return root

In [6]:
import time

# Function to process each book and build the binary search tree
def process_books(books):
    start_time = time.time()

    # Construct the binary search tree
    bst = construct_bst(books)

    elapsed_time = time.time() - start_time
    print(f"Indexing completed in {elapsed_time} seconds")

    return bst

# Process the books and build the binary search tree
bst = process_books(shakespeare_books)

Indexing completed in 33.65950107574463 seconds


In [23]:
def parse_query(query):
    # Split the query into words and remove any whitespace
    tokens = query.split()

    # Stack to handle parentheses
    stack = []
    # List to store the parsed query
    parsed_query = []

    for token in tokens:
        # Remove any parentheses from the token
        token = token.strip('()')

        # If the token is an operator, pop from the stack until reaching an operator with lower precedence
        while stack and stack[-1] in ('AND', 'OR') and (token in ('AND', 'OR') and (token_priority(token) <= token_priority(stack[-1]))):
            parsed_query.append(stack.pop())
        stack.append(token)

    # Pop any remaining operators from the stack
    while stack:
        parsed_query.append(stack.pop())

    return parsed_query

def token_priority(token):
    # Assign priority to operators (lower value means higher priority)
    if token == 'AND':
        return 1
    elif token == 'OR':
        return 0
    return -1  # Non-operators

In [19]:
import os

class TreeNode:
    def __init__(self, term, books):
        self.term = term
        self.books = books
        self.left = None
        self.right = None

# Function to search for a word in the binary search tree
def search_word(bst, term):
    # Check if the term is an operator (e.g., OR, AND)
    if term.upper() in ('OR', 'AND'):
        # For simplicity, return an empty list for operators
        return []

    node = search_word_recursive(bst, term)
    if node:
        print(f"Found term '{term}' in documents: {node.books}")
        return node.books
    else:
        print(f"Term '{term}' not found in any document")
        return None



def search_word_recursive(node, term):
    if node is None:
        return None

    if term == node.term:
        return node

    if term < node.term:
        return search_word_recursive(node.left, term)
    else:
        return search_word_recursive(node.right, term)


In [28]:
# Function to apply query rules (AND, OR)
def apply_query_rules(result_docs, parsed_query):
    if not result_docs:
        return []

    # Initialize the final result as the first set of documents
    final_result = set(result_docs[0])

    # Iterate through the parsed query
    i = 0
    while i < len(parsed_query):
        term = parsed_query[i]

        if term.upper() == 'AND':
            # Apply AND operation
            i += 1
            next_term = parsed_query[i]
            next_docs = set(search_word(bst, next_term))
            final_result.intersection_update(next_docs)

        elif term.upper() == 'OR':
            # Apply OR operation
            i += 1
            next_term = parsed_query[i]
            next_docs = set(search_word(bst, next_term))
            final_result.update(next_docs)

        i += 1

    return list(final_result)

In [29]:
import time

def search_query(bst, query):
    # Parse the query
    parsed_query = parse_query(query)

    # Measure the start time
    start_time = time.time()

    # Search for each word in the tree
    result_docs = []
    for term in parsed_query:
        docs = search_word(bst, term)
        if docs is not None:
            result_docs.extend(docs)
        else:
            # If the word is not found, find the closest word using Levenshtein algorithm
            closest_word = find_closest_word(bst, term)
            if closest_word:
                print(f"Word '{term}' not found. Did you mean '{closest_word}'?")

    # Apply query rules (AND, OR)
    result_docs = apply_query_rules(result_docs, parsed_query)

    # Measure the elapsed time
    elapsed_time = time.time() - start_time
    print(f"Query executed in {elapsed_time:.6f} seconds")

    return result_docs

In [26]:
from nltk.metrics.distance import edit_distance

# Function to search for a word in the binary search tree
def search_word(bst, term):
    # Check if the term is an operator (e.g., OR, AND)
    if term.upper() in ('OR', 'AND'):
        # For simplicity, return an empty list for operators
        return []

    node = search_word_recursive(bst, term)
    if node:
        print(f"Found term '{term}' in documents: {node.books}")
        return node.books
    else:
        print(f"Term '{term}' not found in any document")

        # Find the closest word using Levenshtein distance
        closest_word = find_closest_word(bst, term)
        if closest_word:
            print(f"Did you mean '{closest_word}'?")
        return None


In [32]:
def find_closest_word(bst, term):
    # Initialize variables for the closest word
    closest_word = [None]
    min_distance = [float('inf')]

    # Function to calculate Levenshtein distance between two strings
    def levenshtein_distance(s1, s2):
        if len(s1) < len(s2):
            return levenshtein_distance(s2, s1)

        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

    # Traverse the tree and find the closest word
    traverse_for_closest_word(bst, term, levenshtein_distance, closest_word, min_distance)

    return closest_word[0]

def traverse_for_closest_word(node, term, distance_function, closest_word, min_distance):
    if node is not None:
        distance = distance_function(term, node.term)
        if distance < min_distance[0]:
            min_distance[0] = distance
            closest_word[0] = node.term

        if term < node.term:
            traverse_for_closest_word(node.left, term, distance_function, closest_word, min_distance)
        elif term > node.term:
            traverse_for_closest_word(node.right, term, distance_function, closest_word, min_distance)
        else:
            traverse_for_closest_word(node.right, term, distance_function, closest_word, min_distance)


In [12]:
# Add debug prints in the process_books function
def process_books(books):
    root = None

    for book_path in books:
        with open(book_path, 'r', encoding='utf-8') as file:
            content = file.read()

        words = normalize_text(content)

        for word in words:
            root = insert_word(root, word, book_path)

    print("BST after processing books:", root)

    return root

In [33]:
# Process the books and build the binary search tree
bst = process_books(shakespeare_books)

# Example query
user_query = "(king OR queen) AND (hamlet OR macbeth)"
result_documents = search_query(bst, user_query)

# Print the result documents
print("Result Documents:")
for doc in result_documents:
    print(os.path.basename(doc))

BST after processing books: <__main__.TreeNode object at 0x7e3b1e782d40>
Found term 'macbeth' in documents: ['/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGED

In [34]:
# Example queries
query1 = "(king OR queen) AND (hamlet OR macbeth)"
query2 = "(love OR hate) AND (romeo OR juliet)"
query3 = "sonnet AND (18 OR 130)"
query4 = "duke AND (othello OR measure_for_measure)"
query5 = "moon AND (dream OR night)"

# Process the books and build the binary search tree
bst = process_books(shakespeare_books)

# Test each query
for i, query in enumerate([query1, query2, query3, query4, query5], 1):
    print(f"\nExecuting Query {i}: {query}")
    result_documents = search_query(bst, query)

    # Print the result documents
    print("Result Documents:")
    for doc in result_documents:
        print(os.path.basename(doc))


BST after processing books: <__main__.TreeNode object at 0x7e3b1e0487f0>

Executing Query 1: (king OR queen) AND (hamlet OR macbeth)
Found term 'macbeth' in documents: ['/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF MACBETH.txt', '/content/extracted_books/BOOKS/20. THE TRAGEDY OF 



---



---



# **Results:**
**The project successfully processed the Shakespearean books, built an efficient search mechanism, and demonstrated its capabilities through example queries.**

**This Shakespeare Search Engine provides a user-friendly interface for exploring and discovering specific themes, characters, or phrases within the extensive collection of Shakespeare's writings.**